In [1]:
import os
import gc
from joblib import dump, load
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
import lightgbm as lgb

# load files

In [2]:
train = pd.read_parquet("../input/train_full_features.parquet")

# define loss and metric

In [3]:
def amex_metric(y_true, y_pred):
    
    labels = np.transpose(np.array([y_true, y_pred]))
    labels = labels[labels[:, 1].argsort()[::-1]]
    
    weights = np.where(labels[:,0]==0, 20, 1)
    cut_vals = labels[np.cumsum(weights) <= int(0.04 * np.sum(weights))]
    top_four = np.sum(cut_vals[:,0]) / np.sum(labels[:,0])
    gini = [0,0]
    
    for i in [1, 0]:
        labels = np.transpose(np.array([y_true, y_pred]))
        labels = labels[labels[:, i].argsort()[::-1]]
        weight = np.where(labels[:,0]==0, 20, 1)
        weight_random = np.cumsum(weight / np.sum(weight))
        total_pos = np.sum(labels[:, 0] *  weight)
        cum_pos_found = np.cumsum(labels[:, 0] * weight)
        lorentz = cum_pos_found / total_pos
        gini[i] = np.sum((lorentz - weight_random) * weight)
        
    return 0.5 * (gini[1]/gini[0] + top_four)

In [4]:
def lgb_amex_metric(y_pred, y_true):
    y_true = y_true.get_label()
    return "amex_metric", amex_metric(y_true, y_pred), True

# define training config

In [5]:
seed = 42
n_folds = 5

features = load("selected_features.pkl")

target = "target"

cat_features_base = [
    "B_30",
    "B_38",
    "D_114",
    "D_116",
    "D_117",
    "D_120",
    "D_126",
    "D_63",
    "D_64",
    "D_66",
    "D_68"
] 
cat_features = [
    "{}_last".format(feature) for feature in cat_features_base
]
cat_features = [feature for feature in cat_features if feature in features]
            
params = {
    "objective": "binary",
    "metric": "binary_logloss",
    "boosting": "dart",
    "seed": seed,
    "num_leaves": 100,
    "learning_rate": 0.01,
    "feature_fraction": 0.20,
    "bagging_freq": 10,
    "bagging_fraction": 0.50,
    "n_jobs": -1,
    "lambda_l2": 2,
    "min_data_in_leaf": 40,
}

def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    
seed_everything(seed)

In [6]:
target_fold = 2

kfold = StratifiedKFold(
    n_splits=n_folds, 
    shuffle=True, 
    random_state=seed
)

for fold, (trn_ind, val_ind) in enumerate(kfold.split(train, train[target])):
    
    if fold == target_fold:
        break

x_train, x_val = train.loc[trn_ind, features], train.loc[val_ind, features]
y_train, y_val = train.loc[trn_ind, target], train.loc[val_ind, target]

lgb_train = lgb.Dataset(x_train, y_train, categorical_feature=cat_features)
lgb_valid = lgb.Dataset(x_val, y_val, categorical_feature=cat_features)

In [7]:
del train, x_train, x_val, y_train, y_val
gc.collect()

0

In [8]:
save_folder = os.path.join("../ckpt/lgbm_seed_{}".format(seed))
if not os.path.exists(save_folder):
    os.mkdir(save_folder)
    
save_path = os.path.join(save_folder, "fold_{}".format(target_fold))
if not os.path.exists(save_path):
    os.mkdir(save_path)

In [9]:
print("#" * 50)
print("Training fold {} with {} features...".format(target_fold, len(features)))

global max_score 
max_score = 0.793

def save_model():
    def callback(env):
        global max_score
        iteration = env.iteration
        score = env.evaluation_result_list[3][2]
        if iteration % 100 == 0:
            print("iteration {}, score= {:.05f}".format(iteration,score))
        if score > max_score:
            max_score = score
            print("High Score: iteration {}, score={:.05f}".format(iteration, score))
            dump(env.model, os.path.join(save_path, "{:.05f}.pkl".format(score)))

    callback.order = 0
    return callback

model = lgb.train(
    params=params,
    train_set=lgb_train,
    num_boost_round=20000,
    valid_sets = [lgb_train, lgb_valid],
    early_stopping_rounds=100,
    verbose_eval = 500,
    feval = lgb_amex_metric,
    callbacks=[save_model()],
)

##################################################
Training fold 2 with 1950 features...


/home/caozhehan/miniconda3/envs/trade/lib/python3.8/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/home/caozhehan/miniconda3/envs/trade/lib/python3.8/site-packages/lightgbm/basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
/home/caozhehan/miniconda3/envs/trade/lib/python3.8/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Info] Number of positive: 95062, number of negative: 272068


Exception ignored on calling ctypes callback function: <function _log_callback at 0x7f5d29cb7c10>
Traceback (most recent call last):
  File "/home/caozhehan/miniconda3/envs/trade/lib/python3.8/site-packages/lightgbm/basic.py", line 91, in _log_callback
    def _log_callback(msg: bytes) -> None:
KeyboardInterrupt: 
/home/caozhehan/miniconda3/envs/trade/lib/python3.8/site-packages/lightgbm/basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')


Auto-choosing row-wise multi-threading, the overhead of testing was 51.161345 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 211535
[LightGBM] [Info] Number of data points in the train set: 367130, number of used features: 1950


/home/caozhehan/miniconda3/envs/trade/lib/python3.8/site-packages/lightgbm/basic.py:1513: UserWarning: categorical_column in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')


KeyboardInterrupt: 

In [ ]:
# def training(train):
    
#     # create a numpy array to store out of folds predictions
#     oof_predictions = np.zeros(len(train))
    
#     kfold = StratifiedKFold(
#         n_splits=n_folds, 
#         shuffle=True, 
#         random_state=seed
#     )
    
#     for fold, (trn_ind, val_ind) in enumerate(kfold.split(train, train[target])):
        
#         print("#" * 50)
#         print("Training fold {} with {} features...".format(fold, len(features)))
        
#         x_train, x_val = train.loc[trn_ind, features], train.loc[val_ind, features]
#         y_train, y_val = train.loc[trn_ind, target], train.loc[val_ind, target]
        
#         lgb_train = lgb.Dataset(x_train, y_train, categorical_feature=cat_features)
#         lgb_valid = lgb.Dataset(x_val, y_val, categorical_feature=cat_features)
#         model = lgb.train(
#             params=params,
#             train_set=lgb_train,
#             num_boost_round=10500,
#             valid_sets = [lgb_train, lgb_valid],
#             early_stopping_rounds=100,
#             verbose_eval = 500,
#             feval = lgb_amex_metric
#         )
#         # save best model
#         dump(model, "../ckpt/lgbm_{}_{}.pkl".format(fold, seed))
        
#         # predict validation
#         val_pred = model.predict(x_val)
        
#         # add to out of folds array
#         oof_predictions[val_ind] = val_pred
        
#         # compute fold metric
#         score = amex_metric(y_val, val_pred)
#         print("fold {} score is {}".format(fold, score))
        
#         del x_train, x_val, y_train, y_val, lgb_train, lgb_valid
#         gc.collect()
        
#     # compute oof
#     score = amex_metric(train[target], oof_predictions)
#     print("oof score is {}".format(score))
    
#     # create a dataframe to store out of folds predictions
#     oof_df = pd.DataFrame({"customer_ID": train["customer_ID"], "target": train[target], "prediction": oof_predictions})
#     oof_df.to_parquet("lgbm_oof_{}.parquet".format(seed))
    
#     return oof_df